## 나이브베이즈 타이타닉 정확도 코드

In [2]:
import pandas as pd
# 1단계. csv --> 데이터 프레임으로 변환
df = pd.read_csv('d:\\data\\train.csv')

# 컬럼이 모두다 출력될 수 있도록 출력할 열의 개수 한도를 늘리기
pd.set_option('display.max_columns', 15)



# 2.1 결측치를 확인하고 제거하거나 치환한다.
# 타이타닉 데이터 프레임의 자료형을 확인한다.


# 2.2 결측치(NaN)을 확인한다.

# 2.3 deck 컬럼과 Cabin 컬럼을 삭제한다.
rdf = df.drop(['Cabin'], axis =1)


# 2.4 Age(나이) 열에 나이가 없는 모든 행을 삭제한다.
# age 열에 891개의 행중에 177개가 NaN값

rdf = rdf.dropna(subset = ['Age'], how='any', axis = 0)


# 설명
# 데이터가 한개라도 없으면 drop 해라 (how = 'any')
# 모든 데이터가 없으면 drop 해라 (how = 'all')

# 2.5 embark 열의 NaN값을 승선도시중 가장 많이 출현한 값으로 치환하기.

most_freq = rdf['Embarked'].value_counts(dropna=True).idxmax()
rdf['Embarked'].fillna(most_freq, inplace = True)


# 3. 범주형 데이터를 숫자형으로 변환하기

# 3.1 feature selection (분석에 필요한 속성을 선택)
ndf = rdf[['Survived','Pclass','Sex','Age','SibSp','Parch','Embarked']]
print(ndf.head(), '\n')


# 선택된 컬럼중 2개(sex, embarked)가 범주형(문자형)이다.

# 3.2 범주형 데이터를 숫자로 변환(원핫 인코딩)
gender = pd.get_dummies(ndf['Sex'])
ndf = pd.concat([ndf,gender], axis = 1)

onehot_embarked = pd.get_dummies(ndf['Embarked'], prefix = 'town')
ndf = pd.concat([ndf, onehot_embarked], axis = 1)

ndf.drop(['Sex', 'Embarked'], axis = 1 , inplace = True)


# 4단계. 정규화
# survived  pclass   age  sibsp  parch  female  male  town_C  town_Q  town_S
# survived (종속변수 y)
# pclass   age  sibsp  parch  female  male  town_C  town_Q  town_S (독립변수 X)

# 4.1 독립변수와 종속변수(라벨)를 지정한다.
X = ndf[['Pclass','Age','SibSp','Parch','female','male','town_C','town_Q','town_S']] # 독립변수
y = ndf['Survived'] # 종속변수

# 4.2 독립변수들을 정규화(normalization) 한다.
from sklearn import preprocessing

X = preprocessing.StandardScaler().fit(X).transform(X)


# 4.2 독립변수들을 정규화(normalization) 한다.
from sklearn import preprocessing

X = preprocessing.StandardScaler().fit(X).transform(X)


# 5단계 . 데이터셋을 훈련데이터와 테스트 데이터로 나눈다. (반복문까지 추가함!!!!)

u = []
for i in range(20,30):
    
    from sklearn.model_selection import train_test_split
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = i)
    
    # 설명 : test_size = 0.3 에 의해서 7:3 비율로 훈련과 테스트를 나누고 
    # random_state = 10 에 의해서 나중에 split할 때도 항상 일정하게 split 할 수 있게 한다.

    
    # 6단계. 머신러닝 모델을 생성한다.(knn을 사용)
    for j in range(1,10):
        from sklearn.naive_bayes import GaussianNB, BernoulliNB
        
        model = BernoulliNB(alpha= j) # 이걸로 라플라스 값을 지정할 수도 있다.
        model.fit(X_train, y_train)
        
        # 7단계. 테스트 데이터로 예측하기
        
        y_hat = model.predict(X_test)

   
        # 8단계. 모형의 예측 능력을 평가한다.
        from sklearn import metrics
        naive_matrix = metrics.confusion_matrix(y_test,y_hat)
        print('alpha =', j, 'random_state =', i)
        print(naive_matrix)
        
        # 9단계 정확도 확인
        from sklearn.metrics import accuracy_score
        accuracy = accuracy_score( y_test, y_hat)
        print('accuracy는', accuracy)

        u.append(accuracy)
        
        
        # precision(정밀도) 구하는 코드
        from sklearn.metrics import precision_score
        precision = precision_score(y_test,y_hat)

        print('precision은', precision)
 

        # recall(재현율) 구하는 코드
        from sklearn.metrics import recall_score
        recall = recall_score(y_test,y_hat)

        print('recall은', recall)


        # f1-score 구하는 코드
        from sklearn.metrics import f1_score
        f1 = f1_score(y_test,y_hat)

        print('f1-score 은' ,f1)
        print('\n\n')
        

print('\n')
print('최대정확도는 : ', max(u))
print(u.index(max(u)), '번째순서')

   Survived  Pclass     Sex   Age  SibSp  Parch Embarked
0         0       3    male  22.0      1      0        S
1         1       1  female  38.0      1      0        C
2         1       3  female  26.0      0      0        S
3         1       1  female  35.0      1      0        S
4         0       3    male  35.0      0      0        S 

alpha = 1 random_state = 20
[[106  18]
 [ 25  66]]
accuracy는 0.8
precision은 0.7857142857142857
recall은 0.7252747252747253
f1-score 은 0.7542857142857142



alpha = 2 random_state = 20
[[106  18]
 [ 25  66]]
accuracy는 0.8
precision은 0.7857142857142857
recall은 0.7252747252747253
f1-score 은 0.7542857142857142



alpha = 3 random_state = 20
[[106  18]
 [ 25  66]]
accuracy는 0.8
precision은 0.7857142857142857
recall은 0.7252747252747253
f1-score 은 0.7542857142857142



alpha = 4 random_state = 20
[[106  18]
 [ 25  66]]
accuracy는 0.8
precision은 0.7857142857142857
recall은 0.7252747252747253
f1-score 은 0.7542857142857142



alpha = 5 random_state = 20
[[106  1

f1-score 은 0.6666666666666666



alpha = 3 random_state = 27
[[95 24]
 [36 60]]
accuracy는 0.7209302325581395
precision은 0.7142857142857143
recall은 0.625
f1-score 은 0.6666666666666666



alpha = 4 random_state = 27
[[95 24]
 [36 60]]
accuracy는 0.7209302325581395
precision은 0.7142857142857143
recall은 0.625
f1-score 은 0.6666666666666666



alpha = 5 random_state = 27
[[95 24]
 [36 60]]
accuracy는 0.7209302325581395
precision은 0.7142857142857143
recall은 0.625
f1-score 은 0.6666666666666666



alpha = 6 random_state = 27
[[95 24]
 [36 60]]
accuracy는 0.7209302325581395
precision은 0.7142857142857143
recall은 0.625
f1-score 은 0.6666666666666666



alpha = 7 random_state = 27
[[95 24]
 [36 60]]
accuracy는 0.7209302325581395
precision은 0.7142857142857143
recall은 0.625
f1-score 은 0.6666666666666666



alpha = 8 random_state = 27
[[95 24]
 [36 60]]
accuracy는 0.7209302325581395
precision은 0.7142857142857143
recall은 0.625
f1-score 은 0.6666666666666666



alpha = 9 random_state = 27
[[95 24]
 [36 60]]
ac

## 반복문 x, 나이브 베이즈 코드 (submission.csv제출용 코드)

In [ ]:
import pandas as pd


df = pd.read_csv("d:\\data\\tat\\train.csv")
df2 = pd.read_csv("d:\\data\\tat\\test.csv")

pd.set_option('display.max_columns', 15)


mask1 = (df['Age'] < 10) | (df['Sex'] == 'female')

df['child_women'] = mask1.astype(int)



print(df.info())
print(df2.info())
print('\n')

print(df.isnull().sum(axis=0), '\n')
print(df2.isnull().sum(axis=0), '\n')

rdf = df.drop(['Cabin'], axis =1)
rdf2 = df2.drop(['Cabin'], axis =1)

print(rdf.columns.values, '\n')
print(rdf2.columns.values, '\n')

most_mean = rdf2['Age'].mean()
rdf['Age'].fillna(most_mean, inplace = True)
rdf2['Age'].fillna(most_mean, inplace = True)
print(len(rdf), '\n')
print(rdf2, '\n')



most_freq = rdf['Embarked'].value_counts(dropna=True).idxmax()
most_freq2 = rdf2['Fare'].value_counts(dropna=True).idxmax()
rdf['Embarked'].fillna(most_freq, inplace = True)
rdf2['Fare'].fillna(most_freq2, inplace = True)

print(rdf.isnull().sum(axis=0), '\n')
print(rdf2.isnull().sum(axis=0), '\n')


ndf = rdf[['Survived','Pclass','Sex','Age','SibSp','Parch','Embarked']]
ndf2 = rdf2[['Pclass','Sex','Age','SibSp','Parch','Embarked']]

print(ndf.head(), '\n')
print(ndf2.head(), '\n')

gender = pd.get_dummies(ndf['Sex'])
gender2 = pd.get_dummies(ndf2['Sex'])

ndf = pd.concat([ndf,gender], axis = 1)
ndf2 = pd.concat([ndf2,gender2], axis = 1)
#
onehot_embarked = pd.get_dummies(ndf['Embarked'], prefix = 'town') # 접두사를 정함.
onehot_embarked2 = pd.get_dummies(ndf2['Embarked'], prefix = 'town') # 접두사를 정함.

ndf = pd.concat([ndf, onehot_embarked], axis = 1)
ndf2 = pd.concat([ndf2, onehot_embarked2], axis = 1)

ndf.drop(['Sex', 'Embarked'], axis = 1 , inplace = True)
ndf2.drop(['Sex', 'Embarked'], axis = 1 , inplace = True)

print(ndf.head())
print('\n')

print(ndf2.head())
print('\n')

X = ndf[['Pclass','Age','SibSp','Parch','female','male','town_C','town_Q','town_S']] # 독립변수
y = ndf['Survived'] # 종속변수

test = ndf2[['Pclass','Age','SibSp','Parch','female','male','town_C','town_Q','town_S']]

from sklearn import preprocessing

X = preprocessing.StandardScaler().fit(X).transform(X)
test = preprocessing.StandardScaler().fit(test).transform(test)

print(X)
print(test)

# 5단계 . 데이터셋을 훈련데이터와 테스트 데이터로 나눈다.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 94)



from sklearn.naive_bayes import GaussianNB, BernoulliNB

# 모형 객체 생성
model = BernoulliNB(alpha=100) # 이걸로 라플라스 값을 지정할 수도 있다.
model.fit(X_train, y_train) # model.fit으로 훈련시킨다.


y_hat = model.predict(X_test)


# 정확도 : (109 + 65) / (109 + 16 + 25 + 65)
# 정확도 100%는 현실적으로 나오기 힘드니 (예측: 생존 , 실제: 사망 데이터인 25를 0으로 만드는 것을 궁극적으로 추구한다.)

# confusion matrix에 대한 변수 지정 (tn,fp,fn,tp)
from sklearn import metrics
tn, fp, fn, tp = metrics.confusion_matrix( y_test, y_hat ).ravel()
print(tn,fp,fn,tp)
print('\n')


# 9단계 정확도 확인
from sklearn.metrics import accuracy_score
accuracy = accuracy_score( y_test, y_hat)
print(accuracy)


sample_submission=pd.read_csv('d:\\data\\tat\\gender_submission.csv', index_col=0)

# Prediction
y_pred = model.predict(test)
print(y_pred)

# Submission
submission = pd.DataFrame(data=y_pred,columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('d:\\data\\tat\\submission.csv', index=True)